In [3]:
import json
import os
from PIL import Image
import numpy as np

from create_annotations import create_sub_masks, create_sub_mask_annotation, create_image_annotation, absolute_file_paths

# Label ids of Updated Ecole Centrale Paris (ECP) Facades Dataset 
window_id = 1
wall_id = 2
balcony_id = 3
door_id = 4
roof_id = 5
sky_id = 6
shop_id = 7
chimney_id = 8
outlier_id = 0

# Define which colors match which categories in the images
category_ids = {
    '(255, 0, 0)': window_id, 
    '(255, 255, 0)': wall_id,
    '(128, 0, 255)': balcony_id,
    '(255, 128, 0)': door_id,
    '(0, 0, 255)': roof_id,
    '(128, 255, 255)': sky_id,
    '(0, 255, 0)': shop_id,
    '(128, 128, 128)': chimney_id,
    '(0, 0, 0)': outlier_id    
}

# Create the annotations (Coco format)
coco_format = {
    "images": [
        {
        }
    ],
    "categories": [
        {
            "supercategory": "window",
            "id": 1,
            "name": 'window'
        },
        {
            "supercategory": "wall",
            "id": 2,
            "name": 'wall'
        },
        {
            "supercategory": "balcony",
            "id": 3,
            "name": 'balcony'
        },
        {
            "supercategory": "door",
            "id": 4,
            "name": 'door'
        },
        {
            "supercategory": "roof",
            "id": 5,
            "name": 'roof'
        },
        {
            "supercategory": "sky",
            "id": 6,
            "name": 'sky'
        },
        {
            "supercategory": "shop",
            "id": 7,
            "name": 'shop'
        },
        {
            "supercategory": "chimney",
            "id": 8,
            "name": 'chimney'
        },
        {
            "supercategory": "outlier",
            "id": 0,
            "name": 'outlier'
        },
        
    ],
    "annotations": [
        {
        }
    ]
}

# Get 'images' and 'annotations' info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 1

    annotations = []
    images = []
    
    # Get absolute paths of all files in a directory
    mask_images = absolute_file_paths(maskpath)
    
    for image_id, mask_image in enumerate(mask_images, 1):
        file_name = os.path.basename(mask_image).split('.')[0] + ".jpg"

        # image shape
        mask_image_open = Image.open(mask_image)
        w, h = mask_image_open.size
        
        # 'images' info 
        image = create_image_annotation(file_name, w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, w, h)
        for color, sub_mask in sub_masks.items():
            category_id = category_ids[color]

            # 'annotations' info
            polygons = create_sub_mask_annotation(sub_mask)

            for i in range(len(polygons)):
                min_x, min_y, max_x, max_y = polygons[i].bounds
                width = max_x - min_x
                height = max_y - min_y
                bbox = (min_x, min_y, width, height)
                area = polygons[i].area

                segmentation = np.array(polygons[i].exterior.coords).ravel().tolist()
        
                annotation = {
                    'segmentation': [segmentation],
                    'area': area,
                    'iscrowd': 0,
                    'image_id': image_id,
                    'bbox': bbox,
                    'category_id': category_id,
                    'id': annotation_id
                }

                annotations.append(annotation)
                annotation_id += 1
        
    return images, annotations

if __name__ == '__main__':
    for keyword in ['train', 'val']:
        mask_path = 'dataset/{}_mask'.format(keyword)
        coco_format['images'], coco_format['annotations'] = images_annotations_info(mask_path)
        #print(json.dumps(coco_format))
        with open('output/{}.json'.format(keyword),'w') as outfile:
            json.dump(coco_format, outfile)